In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Load the dataset
df = pd.read_csv('metrike/triton_rezultati.csv')
df

,test,model,batch_size,floating_point,concurrency,kv_cache,Concurrency,Inferences/Second,Client Send,Network+Server Send/Recv,...,Server Compute Input,Server Compute Infer,Server Compute Output,Client Recv,p50 latency,p90 latency,p95 latency,p99 latency,request/response,response wait
0,alpha,gemma3,1,fp16,1:24:2,0,1,0.643508,50,719,...,17,1550278,39,0,1542649,1545638,1546397,1561484,50,1551326
1,alpha,gemma3,2,fp16,1:24:2,0,19,1.185160,54,819,...,66,6588780,81,0,32932954,32973698,32975293,34715581,54,30954911
2,alpha,gemma3,2,fp16,1:24:2,0,11,1.203680,48,752,...,41,6208113,74,0,18114342,18157300,19758219,19796982,48,18116455
3,alpha,gemma3,2,fp16,1:24:2,0,5,1.203680,51,815,...,34,5601306,73,0,8238339,8254527,8257616,8259402,51,8238383
4,alpha,gemma3,2,fp16,1:24:2,0,9,1.203680,54,820,...,33,5777241,83,0,14816465,16490745,16580923,18095577,54,14707821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,kappa,rt-meta-llama,8,f32,1:128:16,0,49,3754.660000,86,4705,...,25,11155,3780,82483,81683,228828,234515,244643,82569,17868
445,kappa,rt-meta-llama,8,f32,1:128:16,0,65,3838.240000,139,3336,...,25,11742,3709,112443,110915,271311,295479,307773,112582,20705
446,kappa,rt-meta-llama,8,f32,1:128:16,0,97,3867.420000,244,5185,...,26,12111,3734,167486,198150,348528,357235,383731,167730,29346
447,kappa,rt-meta-llama,8,f32,1:128:16,0,81,3922.710000,216,4862,...,24,12203,3737,129274,144325,313809,328600,340716,129490,32031


In [58]:
### concatenate test, batch_size, floating_point, kv_cache in one column
df['test'] = (df['test'].astype(str) + '_b' + df['batch_size'].astype(str) + '_' + df['floating_point'].astype(str) +
              '_kv' + df['kv_cache'].astype(str))
df.head()

,test,model,batch_size,floating_point,concurrency,kv_cache,Concurrency,Inferences/Second,Client Send,Network+Server Send/Recv,...,Server Compute Input,Server Compute Infer,Server Compute Output,Client Recv,p50 latency,p90 latency,p95 latency,p99 latency,request/response,response wait
0,alpha_b1_fp16_kv0,gemma3,1,fp16,1:24:2,0,1,0.643508,50,719,...,17,1550278,39,0,1542649,1545638,1546397,1561484,50,1551326
1,alpha_b2_fp16_kv0,gemma3,2,fp16,1:24:2,0,19,1.185160,54,819,...,66,6588780,81,0,32932954,32973698,32975293,34715581,54,30954911
2,alpha_b2_fp16_kv0,gemma3,2,fp16,1:24:2,0,11,1.203680,48,752,...,41,6208113,74,0,18114342,18157300,19758219,19796982,48,18116455
3,alpha_b2_fp16_kv0,gemma3,2,fp16,1:24:2,0,5,1.203680,51,815,...,34,5601306,73,0,8238339,8254527,8257616,8259402,51,8238383
4,alpha_b2_fp16_kv0,gemma3,2,fp16,1:24:2,0,9,1.203680,54,820,...,33,5777241,83,0,14816465,16490745,16580923,18095577,54,14707821


In [59]:
#Zadrži samo relevantne kolone\n",
df_filtered = df[
 ['test', 'model', 'batch_size', 'floating_point', 'kv_cache',
  'Concurrency', 'Inferences/Second',
'p50 latency', 'p90 latency', 'p95 latency', 'p99 latency', 'response wait']].copy()

# convert latency columns to seconds from microseconds
df_filtered['p50 latency'] = round(df_filtered['p50 latency'] / 1000000, 3)
df_filtered['p90 latency'] = round(df_filtered['p90 latency'] / 1000000, 3)
df_filtered['p95 latency'] = round(df_filtered['p95 latency'] / 1000000, 3)
df_filtered['p99 latency'] = round(df_filtered['p99 latency'] / 1000000, 3)

In [61]:
# Add a combined metric
df_filtered['throughput_latency_ratio'] = df_filtered['Inferences/Second'] / df_filtered['p99 latency']
# For each test and model, get the row with the highest ratio
best_rows = df_filtered.loc[df_filtered.groupby(['test', 'model'])['throughput_latency_ratio'].idxmax()]

In [62]:
# order by highest throughput_latency_ratio
best_rows = best_rows.sort_values(by='throughput_latency_ratio', ascending=False)
# Display the best rows
best_rows.reset_index(drop=True, inplace=True)
best_rows.head(10)

,test,model,batch_size,floating_point,kv_cache,Concurrency,Inferences/Second,p50 latency,p90 latency,p95 latency,p99 latency,response wait,throughput_latency_ratio
0,iota_b1_f32_kv8192,rt-mistral,1,f32,8192,113,5969.2200,0.019,0.022,0.022,0.023,16162,259531.304348
1,iota_b4_f32_kv8192,rt-mistral,4,f32,8192,17,4654.9900,0.014,0.017,0.018,0.020,11997,232749.500000
2,kappa_b8_f32_kv0,rt-mistral,8,f32,0,17,6109.4200,0.022,0.024,0.025,0.027,18963,226274.814815
3,iota_b8_f32_kv8192,rt-mistral,8,f32,8192,17,6115.9900,0.022,0.025,0.026,0.028,17683,218428.214286
4,theta_b8_f32_kv4096,rt-mistral,8,f32,4096,17,5347.0600,0.026,0.032,0.033,0.035,14652,152773.142857
5,iota_b4_f32_kv8192,rt-meta-llama,4,f32,8192,17,2906.7600,0.022,0.030,0.033,0.041,13833,70896.585366
6,iota_b1_f32_kv8192,rt-meta-llama,1,f32,8192,65,2654.5200,0.024,0.033,0.037,0.041,15414,64744.390244
7,kappa_b8_f32_kv0,rt-meta-llama,8,f32,0,17,2976.1300,0.033,0.080,0.084,0.091,14962,32704.725275
8,iota_b8_f32_kv8192,rt-meta-llama,8,f32,8192,1,346.1320,0.021,0.033,0.034,0.035,14150,9889.485714
9,iota_b1_f32_kv8192,rt-gemma3,1,f32,8192,1,13.4986,0.074,0.074,0.075,0.077,73426,175.306494


In [63]:
# remove Concurrency column
best_rows.drop(columns=['Concurrency'], inplace=True)
# Display the best rows
best_rows.reset_index(drop=True, inplace=True)
best_rows.head(20)

,test,model,batch_size,floating_point,kv_cache,Inferences/Second,p50 latency,p90 latency,p95 latency,p99 latency,response wait,throughput_latency_ratio
0,iota_b1_f32_kv8192,rt-mistral,1,f32,8192,5969.22000,0.019,0.022,0.022,0.023,16162,259531.304348
1,iota_b4_f32_kv8192,rt-mistral,4,f32,8192,4654.99000,0.014,0.017,0.018,0.020,11997,232749.500000
2,kappa_b8_f32_kv0,rt-mistral,8,f32,0,6109.42000,0.022,0.024,0.025,0.027,18963,226274.814815
3,iota_b8_f32_kv8192,rt-mistral,8,f32,8192,6115.99000,0.022,0.025,0.026,0.028,17683,218428.214286
4,theta_b8_f32_kv4096,rt-mistral,8,f32,4096,5347.06000,0.026,0.032,0.033,0.035,14652,152773.142857
5,iota_b4_f32_kv8192,rt-meta-llama,4,f32,8192,2906.76000,0.022,0.030,0.033,0.041,13833,70896.585366
6,iota_b1_f32_kv8192,rt-meta-llama,1,f32,8192,2654.52000,0.024,0.033,0.037,0.041,15414,64744.390244
7,kappa_b8_f32_kv0,rt-meta-llama,8,f32,0,2976.13000,0.033,0.080,0.084,0.091,14962,32704.725275
8,iota_b8_f32_kv8192,rt-meta-llama,8,f32,8192,346.13200,0.021,0.033,0.034,0.035,14150,9889.485714
9,iota_b1_f32_kv8192,rt-gemma3,1,f32,8192,13.49860,0.074,0.074,0.075,0.077,73426,175.306494


In [64]:
# from the best rows get the best for each model
best_per_model = best_rows.loc[best_rows.groupby('model')['throughput_latency_ratio'].idxmax()]
best_per_model = best_per_model.sort_values(by='throughput_latency_ratio', ascending=False)
# Reset index for better readability
best_per_model.reset_index(drop=True, inplace=True)
# Display the best rows
best_per_model.head(20)

# remove batch size. p50 latency, p90 latency, response wait
best_per_model = best_per_model.drop(columns=['p50 latency', 'p90 latency', 'p95 latency', 'response wait'])
# Display the best rows
best_per_model.reset_index(drop=True, inplace=True)
best_per_model

,test,model,batch_size,floating_point,kv_cache,Inferences/Second,p99 latency,throughput_latency_ratio
0,iota_b1_f32_kv8192,rt-mistral,1,f32,8192,5969.22000,0.023,259531.304348
1,iota_b4_f32_kv8192,rt-meta-llama,4,f32,8192,2906.76000,0.041,70896.585366
2,iota_b1_f32_kv8192,rt-gemma3,1,f32,8192,13.49860,0.077,175.306494
3,beta_b8_bf16_kv0,mistral,8,bf16,0,13.64180,3.165,4.310205
4,eta_b8_f16_kv4096,gemma3,8,f16,4096,4.58020,1.749,2.618754
5,beta_b8_bf16_kv0,meta-llama,8,bf16,0,9.02461,5.139,1.756102
